## Análisis de Particle Swarm para una fuerza de ventas de 6 nodos

Particle Swarm es un algoritmo que recibe los siguientes hiperárametros de entrada: 
+ $\alpha$ (La influencia de la mejor posición individual de cada cada partícula, en la actualización de la posición de cada partícula)
+ $\beta$ (La influencia de la mejor posición global, en la actualización de la posición de cada partícula)
+ *número de iteraciones*
+ *número de partículas*

Los resultados obtenidos pueden verse afectados al variar los valores de tales hiperparámetros. Por otro lado, dependiendo del número de nodos del grafo, estos hiperparámetros también podrían afectar la ruta mínima encontrada por el algoritmo.    

El presente notebook considera la implementación de particle swarm para un grafo con 5 nodos. Como primero objetivo se variarán los hiperparámetros y se identificarán aquellos que den mejores resultados. Entendiéndose como mejores resultados; obtener la ruta con menor distancia. Adicionalmente, una vez seleccionados los mejores hiperparámetros, se correrá el algoritmo 100 veces con el fin de realizar un análisis sobre las rutas obtenidas. Particularmente se tiene interés en revisar variaciones en las rutas obtenidas en cada corrida.

Dentro del conjunto de datos que se tienen disponibles, existen varias fuerzas de venta que deben recorrer 6 nodos. Se decidió elegir la fuerza de venta **80993** perteneciente al estado de Nuevo León para llevar a cabo estas pruebas.

In [1]:
# Librerías

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('../')
from src import Utileria as ut
from src.models import particle_swarm as ps

/home/lauragmz/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### 1. Búsqueda de mejores hiperparámetros

**1.1 Definición de datos de entrada**
+ Grafo completo de los puntos que debe vistar la fuerza de ventas
+ Hiperámetros con los que correrá el algoritmo

In [2]:
# Se obtiene el dataframe que contiene el grafo de la fuerza de venta a evaluar:
str_Query = 'select id_origen, id_destino, distancia from trabajo.grafos where id_fza_ventas={};'

# En el query se especifica el id_fza_venta del cual se quiere obtener su grafo
df_Grafo = ut.get_data(str_Query.format(80993))
df_Grafo

,id_origen,id_destino,distancia
0,11037,1001402004,0.267770992210818
1,11037,1006681965,0.3250713086280114
2,11037,1020053072,2.3599263518563016
3,11037,1020235635,1.2662382942527688
4,11037,1020402992,0.07674597387253217
5,1001402004,1006681965,0.4982193543875845
6,1001402004,1020053072,2.611292644038669
7,1001402004,1020235635,1.5321611628202725
8,1001402004,1020402992,0.23296305988637042
9,1006681965,1020053072,2.141257356651251


In [3]:
# Se crea el diccionario de hiper-parámetros que se evaluarán
dict_Hiper_PS = {'Iteraciones': {10,50, 100},
              'Particulas': {1,5,100},
              'Alfa': {.5, 1},
              'Beta': {.5, 1}
              }

**1.2 Gridsearch**

Dentro de la clase Utileria fue definido un método llamado *GridSearch*, el cual recibe como parámetros el grafo de una fuerza de ventas fijo, un diccionario de parámetros, el algoritmo a evaluar y el número de iteraciones que se correrá por cada combinación de hiperámetros. Este método evalúa el algoritmo con todas las combinaciones que se pueden generar a partir del diccionario de parámetros. En este caso se considerarán 3 valores de *Iteraciones*, 3 valores del *Número de Partículas*, 2 valores de $\alpha$ y 2 de $\beta$; dando lugar a un total de 36 combinaciones. Cada combinación de hiperarámetros se correrá 100 veces y como resultado se obtendrá una tabla indicando los Hiperámetros utilizados, las distancias mínima y máxima obtenidas dentro de las 100 corridas; y el número de corridas en que se repitió tal distancia mínima.

In [4]:
%%time

# Se corre el GridSearch para el grafo y los hiperparámetros previamente definidos

df_Resultado = ut.GridSearch(df_Grafo, ps.ParticleSwarm, dict_Hiper_PS, 100)

CPU times: user 45.9 s, sys: 26.2 ms, total: 45.9 s
Wall time: 45.9 s


In [5]:
# Se muestra el dataframe con los resultados obtenidos de la corrida del GridSearch
pd.options.display.max_colwidth = 100
df_Resultado

,HiperParámetros,Distancia mínima (km),Distancia máxima (km),Frec. rel. dist. min.
0,"{'Iteraciones': 100, 'Particulas': 1, 'Alfa': 0.5, 'Beta': 0.5}",5.604,8.559,3/100
1,"{'Iteraciones': 100, 'Particulas': 1, 'Alfa': 0.5, 'Beta': 1.0}",5.604,8.532,2/100
2,"{'Iteraciones': 100, 'Particulas': 1, 'Alfa': 1.0, 'Beta': 0.5}",5.659,8.559,3/100
3,"{'Iteraciones': 100, 'Particulas': 1, 'Alfa': 1.0, 'Beta': 1.0}",5.604,8.442,1/100
4,"{'Iteraciones': 100, 'Particulas': 100, 'Alfa': 0.5, 'Beta': 0.5}",5.604,5.659,99/100
5,"{'Iteraciones': 100, 'Particulas': 100, 'Alfa': 0.5, 'Beta': 1.0}",5.604,5.659,98/100
6,"{'Iteraciones': 100, 'Particulas': 100, 'Alfa': 1.0, 'Beta': 0.5}",5.604,5.604,100/100
7,"{'Iteraciones': 100, 'Particulas': 100, 'Alfa': 1.0, 'Beta': 1.0}",5.604,5.676,93/100
8,"{'Iteraciones': 100, 'Particulas': 5, 'Alfa': 0.5, 'Beta': 0.5}",5.604,7.519,31/100
9,"{'Iteraciones': 100, 'Particulas': 5, 'Alfa': 0.5, 'Beta': 1.0}",5.604,8.028,35/100


**1.3 Análisis y Resultados**

En primer lugar es importante mencionar las motivaciones de ciertos de los hiperparámetros elegidos. 

+ Con respecto al número de iteraciones, $100$ iteraciones fueron elegidas con base en pruebas realizadas en la literatura [paper](http://ijcsi.org/papers/IJCSI-9-6-2-264-271.pdf), tomando este número como base, se determinó hacer pruebas considerando la mitad de iteraciones ($50$); y finalmente para tener un valor extremo contra el cuál comparar se eligió dismniur el número de iteraciones incial en un $90\%$, dando lugar a $10$ iteraciones.

+ Sobre el número de partículas, la primera idea fue considerar tantas partículas como número de nodos, sin embargo se están considerando $5$; y tomar un número de partículas considerablemente mayor ($100$) y uno considerablemente menor ($1$). En este caso, al tener únicamente $6$ nodos no es posible tomar un número menor de partículas extremo; pero los resultados obtenidos con $1$ partícula aportan datos muy relevantes para el análisis.

+ Puesto que $\alpha$ y $\beta$ representan probabilidades, los valores que pueden tomar están entre $0$ y $1$. De manera análoga a los otros parámetros, se consideran valores medios($0.5$) y altos($1$).

Para poder interpretar los resultados mostrados en el dataframe anterior es conveniente recordar que se realizaron 36 pruebas, es decir, se tuvieron 36 combinaciones de hiperparámetros; y cada una de esas combinaciones se corrió $100$ veces, dando un total de $3,600$ corridas. En las $100$ corridas de cada prueba se registraron la distancia mínima obtenida, la distancia máxima y la frecuencia relativa de la distancia mínima. A continuación se enlistan observaciones importantes derivadas de estas pruebas:

+ La distancia mínima obtenida: $5.604 km$ es la misma para las $36$ combinaciones; sin embargo la distancia máxima obtenida varía entre $5.604 km$ y $8.559 km$. Esto demuestra que apesar de utilizar los mismos hiperparámetros, el algoritmo puede dar resultados distintos en cada corrida.

+ Al considerar únicamente $1$ partícula y variar el resto de los hiperparámetros, se observa que las frecuencias relativas de la distancia mínima en cada una de las pruebas ($100$ corridas por prueba) son menores al $5\%$; las cuales representan las frecuencias relativas más bajas de las 36 pruebas. 

+ Al fijar el número de partículas en $100$ y variar el resto de los hiperparámetros, se obtienen frecuencias relativas de la distancia mínima mayores al $90\%$. Éstas corresponden a las frecuencias relativas más altas de las 36 pruebas realizadas.

+ En general, al fijar tanto el número de iteraciones como el número de partículas y variar los valores de $\alpha$ y $\beta$, se observa que bajo la siguiente condición $\alpha = \beta = 1$ se obtiene los valores más chicos de frecuencias relativas de la distancia mínima. Al tomarr estos parámetros igual a $1$ se está considerando que ambos influyen "totalmente" en la actualización de la posición de cada una de las partículas; ya que $\alpha$ y $\beta$ representan la influencia de actualización local y global respectivamente. Bajo estas condiciones (número de iteraciones de partículas fijas), se obtuvieron valores de frecuencia relativa más altos cuando $\alpha=1$ y $\beta=0.5$; esto significa que tiene mayor peso la influencia de la posición individual de cada partícula en la actualización ($\alpha$) de cada iteración.

+ Con respecto al número de iteraciones, puede decirse que este parámetro depende en gran medida del número de partículas. Por un lado, al considerar $100$ iteraciones y $1$ ó $5$ partículas, las frecuencias relativas de la distancia mínima son bajas. Por otro lado, al disminuir el número de iteraciones a $10$ y aumentar el número de partículas a $100$, tales frecuencias relativas son las más altas. 

### 2. Análisis de rutas

Con base en los resultados anteriores, es posible elegir hiperparámetros que maximen la frecuencia relativa de la distancia mínima. Los considerados como mejores hiperparámetros se desglosan en seguida:

+ Número de iteraciones: 100
+ Número de partículas: 100
+ $\alpha$: 1
+ $\beta$: 0.5

Fijando los valores anteriores, se correrá el algoritmo de Particle Swarm 100 veces para la fuerza de ventas con 6 nodos definida al inicio del notebook; y se llevará a cabo un análisis sobre las distintas rutas que arroja el algoritmo, así como la distancia mínima obtenida para tales rutas.

In [16]:
# Definición de dataframe donde se almacenarán las rutas
rutas = pd.DataFrame(index=range(100),columns=['Distancia', 'Ruta'])

# Definición de hiperparámetros
dict_Hiper = {'Iteraciones': 10,
              'Particulas': 100,
              'Alfa': 1,
              'Beta': .5
              }

In [17]:
%%time

# 100 ejecuciones de particle swarm para una fuerza de ventas con 5 nodos,
# considerando los mejores hiperparámetros

for corrida in range(100):
    
    PS = ps.ParticleSwarm(df_Grafo,dict_Hiper)
    PS.Ejecutar()
    
    min_distancia = round(PS.nbr_MejorCosto,3)
    mejor_ruta = ut.convert(PS.lst_MejorCamino)
    
    rutas.Distancia[corrida] = min_distancia
    rutas.Ruta[corrida] = mejor_ruta

CPU times: user 844 ms, sys: 3.12 ms, total: 847 ms
Wall time: 857 ms


In [18]:
# Primeras 7 rutas obtenidas
rutas.head(7)

,Distancia,Ruta
0,5.604,11037-1020235635-1020053072-1006681965-1001402...
1,5.604,11037-1020235635-1020053072-1006681965-1001402...
2,5.604,11037-1020402992-1001402004-1006681965-1020053...
3,5.604,11037-1020402992-1001402004-1006681965-1020053...
4,5.604,11037-1020402992-1001402004-1006681965-1020053...
5,5.604,11037-1020235635-1020053072-1006681965-1001402...
6,5.604,11037-1020235635-1020053072-1006681965-1001402...


In [22]:
# Descripción de las rutas obtenidas

rutas.groupby('Ruta').describe()

Distancia                \
                                                       count unique    top   
Ruta                                                                         
11037-1006681965-1020053072-1020235635-10204029...         1      1  5.659   
11037-1020235635-1020053072-1006681965-10014020...        57      1  5.604   
11037-1020402992-1001402004-1006681965-10200530...        42      1  5.604   

                                                         
                                                   freq  
Ruta                                                     
11037-1006681965-1020053072-1020235635-10204029...    1  
11037-1020235635-1020053072-1006681965-10014020...   57  
11037-1020402992-1001402004-1006681965-10200530...   42

La tabla anterior evidencia que de las $100$ veces que se ejecutó el algoritmo se obtuvieron $3$ rutas distintas; $2$ de ellas con la misma distancia mínima de $5.604 km$ y la tercera con una distancia de $5.659 km$. La variación entres estas dos distancias es de $55m$. Es importante recalcar que en un $99\%$ de las ejecuciones se obtuvo la menor distancia.

Este resultado demuestra que el algoritmo de Particle Swarm puede arrojar rutas distintas con el mismo costo (distancia mínima). Por otro lado también es posible que arroje otras rutas cuya distancia no es la mínima distancia obtenida durante todas las simulaciones.  

### Conclusiones:
+ Los resultados del algoritmo pueden variar en cada corrida, aún cuando se mantengan fijos los hiperparámetros.
+ Los mejores hiperparámetros obtenidos para 5 nodos consideran $100$ iteraciones y $100$ partículas, $\alpha = 1$ y $\beta = 0.5$.
+ Correr el algoritmo una sola vez no garantiza que se obtenga la distancia mínima; sin embargo al considerar un número de partículas alrededor de $100$ aumenta la probabilidad de encontrar la distancia mínima bajo esta condición.
+ Al ejecutar el algoritmo varias veces, es posible obtener distintas rutas cuya distancia sea la misma.